To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**NEW** Unsloth now supports training the new **gpt-oss** model from OpenAI! You can start finetune gpt-oss for free with our **[Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)**!

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


## Setting up the Environment

This section prepares the Google Colab environment for fine-tuning the Mistral model using Unsloth. It includes instructions for running the notebook and links to helpful resources like the Unsloth website, Discord, and documentation.

### Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from google.colab import drive

# Check if drive is already mounted
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")

pdf_File_Path = '/content/drive/MyDrive/HealthcareCJM/TuningPart2/Referral_Letter_Events.pdf'
input_File = '/content/drive/MyDrive/HealthcareCJM/TuningPart2/Referral_Letter_Events.docx'
pdf_Output_Path = '/content/drive/MyDrive/HealthcareCJM/TuningPart2/Referral Letter_Doc.docx'
Model_Output_BF_Tuned = '/content/drive/MyDrive/HealthcareCJM/TuningPart2/Model_Output_BF_Tuned.xlsx'
Model_Output_AF_Tuned = '/content/drive/MyDrive/HealthcareCJM/TuningPart2/Model_Output_AF_Tuned.xlsx'

Mounted at /content/drive


In [ ]:
!pip install PyMuPDF python-docx langchain

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

pdf_text = extract_text_from_pdf(pdf_File_Path)
print(f"Extracted {len(pdf_text)} characters from PDF.")

Extracted 15942 characters from PDF.


In [ ]:
from docx import Document

def save_text_to_docx(text, output_path="output.docx"):
    doc = Document()
    for paragraph in text.split('\n\n'):
        doc.add_paragraph(paragraph.strip())
    doc.save(pdf_Output_Path)
    print(f"Saved text to {pdf_Output_Path}")

save_text_to_docx(pdf_text)

Saved text to /content/drive/MyDrive/HealthcareCJM/TuningPart2/Referral Letter_Doc.docx


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Treat the full PDF text as one document
from langchain_core.documents import Document as LC_Document
doc = LC_Document(page_content=pdf_text)

# Chunk it
page_chunks = splitter.split_documents([doc])
print(f"Created {len(page_chunks)} chunks.")

Created 42 chunks.


In [ ]:
mistral_system_prompt = """
You are an AI system specialized in extracting structured clinical events for Healthcare Journey Mapping.

Your goal: From the given text chunk, extract all clinical events mentioned and represent them in a structured, chronological manner.

### Instructions:
1. Identify events such as:
   - Diagnosis
   - Referral Sent
   - Specialist Visit
   - Hospitalization
   - Procedure
   - Imaging Ordered / Imaging Result
   - Lab Ordered / Lab Result
   - Medication Started / Changed / Stopped
   - Follow-up Visit
   - Missed Appointment
   - Patient Reported Symptom
   - Functional/Social Note
   - Other relevant clinical events

2. For each event, provide:
   - **event_type**: One of the categories above.
   - **date**: Use `YYYY-MM-DD` if exact date is available; otherwise `YYYY-MM` or "Unknown".
   - **provider**: Doctor name or facility if present; otherwise "N/A".
   - **details**: Short summary (e.g., reason, findings, medication name/dose, lab value).

3. Do NOT infer or hallucinate events not explicitly mentioned in the text.
4. Keep identifiers exactly as in text (do not generate new names).
5. Output strictly as a JSON array.

### JSON Format:
[
  {
    "event_type": "",
    "date": "",
    "provider": "",
    "details": ""
  }
]

### Example:
[
  {
    "event_type": "Referral Sent",
    "date": "2023-03-01",
    "provider": "Dr. Smith",
    "details": "Concern: nephropathy"
  },
  {
    "event_type": "Hospitalization",
    "date": "2024-02-10",
    "provider": "St. Mary’s Hospital",
    "details": "Acute kidney injury, creatinine peaked at 3.0 mg/dL"
  }
]

Now process this text chunk and return ONLY valid JSON with all events found:
<<CHUNK>>
"""

In [ ]:
# Install dependencies
!pip install -q -U transformers accelerate bitsandbytes

# Import libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# Choose model ID
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

# Optional: configure 4-bit quantization if using limited GPU memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model with quantization and auto device placement
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

# Build a text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.8.10 requires tyro, which is not installed.
unsloth-zoo 2025.8.9 requires msgspec, which is not installed.
unsloth-zoo 2025.8.9 requires tyro, which is not installed.


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


In [ ]:
import json
import pandas as pd
import torch
from datetime import datetime
import time

# ---------- Config ----------
num_chunks_to_process = len(page_chunks)   # or set a smaller number
# num_chunks_to_process = 3
rows = []
output_file = Model_Output_BF_Tuned

# ---------- Helpers ----------
def build_full_prompt(chunk_text: str) -> str:
    return (
        mistral_system_prompt.replace("<<CHUNK>>", chunk_text)
        + "\n\nReturn ONLY a valid JSON array without any text before or after."
    )

def as_raw_text_cell(chunk) -> str:
    # Match your example: Raw Text should begin with page_content='...'
    text = getattr(chunk, "page_content", str(chunk))
    return f"page_content='{text}'"

def extract_json_array(generated: str) -> str:
    """
    Extract the first top-level JSON array from a text string.
    Returns the raw JSON string (pretty printed) or raises.
    """
    cleaned = generated.strip()

    # strip optional code fences
    if cleaned.startswith("```json"):
        cleaned = cleaned[len("```json"):].lstrip()
    if cleaned.endswith("```"):
        cleaned = cleaned[:-3].rstrip()

    start = cleaned.find('[')
    end = cleaned.rfind(']') + 1
    if start == -1 or end <= 0 or start >= end:
        raise ValueError("No JSON array found in model output.")

    # validate JSON & pretty print
    obj = json.loads(cleaned[start:end])
    return json.dumps(obj, ensure_ascii=False, indent=2)

# ---------- Main loop ----------
device = next(model.parameters()).device

for i, chunk in enumerate(page_chunks[:num_chunks_to_process], start=1):
    # Build full prompt inside a single "user" message (works with Mistral chat template)
    full_prompt = build_full_prompt(getattr(chunk, "page_content", str(chunk)))
    messages = [{"role": "user", "content": full_prompt}]

    # Tokenize according to the tokenizer's chat template
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(device)

    t0 = time.perf_counter()
    # Generate deterministically for clean JSON
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs,
            max_new_tokens=1024,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True,
        )
    elapsed_ms = (time.perf_counter() - t0) * 1000.0

    # Slice to only the newly generated tokens (exclude the prompt)
    gen_only_ids = output_ids[0, inputs.shape[-1]:]
    gen_text = tokenizer.decode(gen_only_ids, skip_special_tokens=True)

    # Extract and pretty-print the JSON array; if it fails, keep raw text
    try:
        pretty_json = extract_json_array(gen_text)
        output_cell = pretty_json
    except Exception:
        # Fallback to raw model output if JSON parsing fails
        output_cell = gen_text.strip()

    rows.append({
        "S.No": i,
        "Raw Text": as_raw_text_cell(chunk),
        "Output": output_cell,
        "Latency": elapsed_ms,  # <--- NEW
    })

    # Print the results
    print(f"--- Chunk {i} ---")
    print(f"Raw Text: {as_raw_text_cell(chunk)}")
    print(f"Output: {output_cell}")
    print(f"latency_ms: {elapsed_ms}")
    print("-" * 20)

# ---------- Save exactly 3 columns to Excel ----------
print("Saving results to Excel...")
df = pd.DataFrame(rows, columns=["S.No", "Raw Text", "Output", "Latency"])
df.to_excel(output_file, index=False)
print(f"✅ Saved {len(df)} rows to {output_file}")
print("Processing complete.")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- Chunk 1 ---
Raw Text: page_content='Referral Letter: Nephrology Evaluation for Mr. xxx Halperin
Date: June 15, 2025
From: Dr. L. yyy, MD
To: Nephrology Department
Subject: Referral for advanced evaluation of chronic kidney disease in the context of complex
multi-system comorbidities.
Dear Colleague,
I am referring my long-standing patient, Mr. xxx W. Halperin, a 71-year-old retired school
principal, for nephrology evaluation in light of accelerated decline in renal function, persistent'
Output: [
  {
    "event_type": "Referral Sent",
    "date": "2025-06-15",
    "provider": "Dr. L. yyy, MD",
    "details": "Referral for advanced evaluation of chronic kidney disease in the context of complex multi-system comorbidities."
  },
  {
    "event_type": "Patient Reported Symptom",
    "date": "Unknown",
    "provider": "N/A",
    "details": "Accelerated decline in renal function."
  }
]
latency_ms: 43052.442758000325
--------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- Chunk 2 ---
Raw Text: page_content='principal, for nephrology evaluation in light of accelerated decline in renal function, persistent
proteinuria, and increasing clinical complexity. Mr. XXXHalperinhas been under my care at
Lakeside Family Medicine since 2012 and presents a multifactorial chronic disease picture
dominated by progressive diabetic nephropathy, long-standing hypertension, and cardiovascular
disease.
Mr. XXXHalperinwas first diagnosed with type 2 diabetes mellitus in 2008, following a period'
Output: [
  {
    "event_type": "Diagnosis",
    "date": "2008-00-00",
    "provider": "N/A",
    "details": "Type 2 diabetes mellitus"
  },
  {
    "event_type": "Follow-up Visit",
    "date": "Unknown",
    "provider": "Lakeside Family Medicine",
    "details": "Under care since 2012"
  },
  {
    "event_type": "Referral Sent",
    "date": "Unknown",
    "provider": "Dr. principal",
    "details": "Nephrology evaluation due to accelerated decline in renal function, persistent p

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

--- Chunk 39 ---
Raw Text: page_content='
In December 2025, routine monitoring continued. No urgent findings noted, but patient
remained under observation for renal and cardiovascular risk factors.

In June 2025, routine monitoring continued. No urgent findings noted, but patient remained
under observation for renal and cardiovascular risk factors.

In July 2025, routine monitoring continued. No urgent findings noted, but patient remained
under observation for renal and cardiovascular risk factors.
'
Output: [
  {
    "event_type": "Follow-up Visit",
    "date": "2025-12-Unknown",
    "provider": "N/A",
    "details": "Routine monitoring continued, no urgent findings noted, under observation for renal and cardiovascular risk factors"
  },
  {
    "event_type": "Follow-up Visit",
    "date": "2025-06-Unknown",
    "provider": "N/A",
    "details": "Routine monitoring continued, no urgent findings noted, under observation for renal and cardiovascular risk factors"
  },
  {
    "event

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- Chunk 40 ---
Raw Text: page_content='under observation for renal and cardiovascular risk factors.

In May 2023, his daughter accompanied him for a visit and expressed concern about his
increasing fatigue and reduced appetite.

In September 2025, routine monitoring continued. No urgent findings noted, but patient
remained under observation for renal and cardiovascular risk factors.

In March 2023, patient began reporting muscle cramps during nighttime, potentially
secondary to electrolyte imbalance.
'
Output: [
  {
    "event_type": "Follow-up Visit",
    "date": "2023-05-01",
    "provider": "N/A",
    "details": "Daughter expressed concern about patient's increasing fatigue and reduced appetite"
  },
  {
    "event_type": "Referral Sent",
    "date": "2023-03-01",
    "provider": "N/A",
    "details": "Patient began reporting muscle cramps during nighttime, potentially secondary to electrolyte imbalance"
  },
  {
    "event_type": "Follow-up Visit",
    "date": "2025-09-01",
 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- Chunk 41 ---
Raw Text: page_content='secondary to electrolyte imbalance.

In October 2023, creatinine reached 2.3 mg/dL; GFR dropped below 30. Discussion around
nephrology referral was reinitiated.

In June 2023, urinalysis showed a rise in protein/creatinine ratio to 1.2 g/g, consistent with
worsening proteinuria.

In May 2025, routine monitoring continued. No urgent findings noted, but patient remained
under observation for renal and cardiovascular risk factors.
'
Output: [
  {
    "event_type": "Lab Result",
    "date": "2023-10-DD",
    "provider": "N/A",
    "details": "Creatinine reached 2.3 mg/dL; GFR dropped below 30"
  },
  {
    "event_type": "Lab Result",
    "date": "2023-06-DD",
    "provider": "N/A",
    "details": "Urinalysis showed a rise in protein/creatinine ratio to 1.2 g/g"
  },
  {
    "event_type": "Discussion",
    "date": "2023-10-DD",
    "provider": "N/A",
    "details": "Reinitiated nephrology referral discussion"
  }
]
latency_ms: 18640.830621000532


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Mistral patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# If not already installed in your Colab
# !pip -q install pandas datasets openpyxl

from google.colab import drive
drive.mount('/content/drive')  # ensure Drive is mounted

import json
import pandas as pd
from datasets import Dataset

xlsx_path = "/content/drive/MyDrive/HealthcareCJM/Clinical_Dataset.xlsx"

# 1) Load Excel
df = pd.read_excel(xlsx_path)

# 2) (Optional) ensure 'output' is a JSON string (if it's a list/dict in cells)
def to_json_str(x):
    if isinstance(x, str):
        try:
            json.loads(x)  # already a JSON string
            return x
        except Exception:
            return x
    return json.dumps(x, ensure_ascii=False)

if "output" in df.columns:
    df["output"] = df["output"].apply(to_json_str)

# 3) Convert to HF Dataset
dataset = Dataset.from_pandas(df, preserve_index=False)

# 4) Print first 10 rows
for i in range(min(10, len(dataset))):
    print(dataset[i])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'input': 'Diagnosed with type 2 diabetes in 2008.', 'output': "[{'event_type': 'Diagnosis', 'date': '2008', 'provider': 'N/A', 'details': 'Type 2 diabetes'}]"}
{'input': "Hospitalized in February 2024 for acute kidney injury at St. Mary's Hospital.", 'output': '[{\'event_type\': \'Hospitalization\', \'date\': \'2024-02\', \'provider\': "St. Mary\'s Hospital", \'details\': \'Acute kidney injury\'}]'}
{'input': 'A1c measured at 7.8% in 2008.', 'output': "[{'event_type': 'Lab Result', 'date': '2008', 'provider': 'N/A', 'details': 'A1c 7.8%'}]"}
{'input': 'Urine albumin/creatinine ratio 320 mg/g reported in March 2024.', 'output': "[{'event_type': 'Lab Result', 'date': '2024-03', 'provider': 'N/A', 'details': 'UACR 320 mg/g'}]"}
{'input': 'Renal ultrasound ordered on 2025-06-10 due to persistent proteinuria.', 'output': "[{'event_type': 'Imaging Ordered', 'date'

In [ ]:
print(dataset.column_names)

['input', 'output']


In [ ]:
# 1) Apply Mistral template
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(tokenizer, chat_template="mistral")

# 2) Build "text" column with your formatting function
# IMPORTANT: For dataset.map(...), formatting function must return a dict: {"text": [...]}.
# Example:
import json

def formatting_prompts_func(examples):
    texts = []
    for inp, out in zip(examples["input"], examples["output"]):
        user_content = "" if inp is None else str(inp).strip()
        out_str = json.dumps(out, ensure_ascii=False) if isinstance(out, (dict, list)) else str(out).strip()

        messages = []
        # Optional: include your hardened system prompt if you use one
        # messages.append({"role": "system", "content": system_prompt.strip()})

        messages.append({"role": "user", "content": user_content})
        messages.append({"role": "assistant", "content": out_str})

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,   # we include assistant ourselves
        )
        if tokenizer.eos_token:
            text += tokenizer.eos_token
        texts.append(text)

    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

# 3) Quick sanity check: ensure "[INST]" and "[/INST]" appear in the text
print("Sample text repr:\n", repr(dataset[0]["text"][:500]))


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Sample text repr:
 "<s>[INST] Diagnosed with type 2 diabetes in 2008. [/INST][{'event_type': 'Diagnosis', 'date': '2008', 'provider': 'N/A', 'details': 'Type 2 diabetes'}]</s></s>"


In [ ]:

Hardened_System_Prompt = """You are an AI system specialized in extracting structured clinical events for Healthcare Journey Mapping.

Your goal: From the given text chunk, extract ONLY clinical events that are explicitly stated, in a structured, chronological manner.

CRITICAL RULES:
- Do NOT infer or hallucinate. If it didn’t happen explicitly, omit it.
- Provider: use exact name/facility if present; else "N/A".
- Date: use the most precise truthful format available:
  - YYYY-MM-DD > YYYY-MM > YYYY > "Unknown"
  - Never invent month/day; never use placeholders like "00".
- Details: concise and grounded. Prefer verbatim numeric values; no embellishment.
- Lists of conditions without separate dates/encounters → ONE Diagnosis event (combine items).
- Distinguish state vs action:
  - Numeric values ⇒ Result (Lab Result / Imaging Result).
  - "ordered/plan/schedule" ⇒ Ordered (Lab Ordered / Imaging Ordered).
- Exclude hypothetical or conditional statements unless the action occurred.

Output strictly as a JSON array of:
[
  { "event_type": "", "date": "", "provider": "", "details": "" }
]

Now process this text chunk:
"""




In [ ]:
# Check trainable params
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable: {trainable/1e6:.1f}M / Total: {total/1e9:.2f}B")

Trainable: 41.9M / Total: 3.80B


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,  # For test run, use num_train_epochs=1 for full
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = Model_Output,
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only

# Mistral-style template uses [INST] ... [/INST] then assistant text.
trainer = train_on_responses_only(
    trainer,
    instruction_part = "[INST]",
    response_part    = "[/INST]",
)

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
dataset[0]["text"]

"<s>[INST] Diagnosed with type 2 diabetes in 2008. [/INST][{'event_type': 'Diagnosis', 'date': '2008', 'provider': 'N/A', 'details': 'Type 2 diabetes'}]</s></s>"

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 9 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss,entropy
1,2.257300,0
2,2.562100,No Log
3,1.859700,No Log
4,1.178900,No Log
5,0.755400,No Log
6,0.342300,No Log
7,0.294300,No Log
8,0.192200,No Log
9,0.079700,No Log
10,0.051700,No Log


In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<s><s>[INST] Renal ultrasound on 2025-06-18 showed increased echogenicity. [/INST][{'event_type': 'Imaging Result', 'date': '2025-06-18', 'provider': 'N/A', 'details': 'Renal ultrasound: increased echogenicity'}]</s></s>"

In [ ]:
from transformers import TextStreamer
from datetime import datetime

num_chunks_to_process = 3  # or whatever you want
all_data = []

for i, chunk in enumerate(page_chunks[:num_chunks_to_process]):
    print(f"\n--- Processing chunk {i+1}/{num_chunks_to_process} ---")
    print(f"\n--- Chunk Data {chunk} ---")

    # Prompt
    messages = [
        {
            "role": "user",
            "content": (
                "Extract all events and their corresponding dates from the paragraph below.\n"
                "Return the output as a JSON array with two fields: 'event' and 'date'.\n"
                "Use concise phrasing. Include only entries that have a valid date.\n\n"
                f"Paragraph:\n{chunk.page_content}"
            )
        }
    ]

    # Apply template & send to model
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    output = model.generate(
        input_ids=inputs,
        streamer=text_streamer,
        max_new_tokens=256,
        use_cache=True
    )

    # Decode model output
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)

    print(output_text)



--- Processing chunk 1/3 ---

--- Chunk Data page_content='Referral Letter: Nephrology Evaluation for Mr. xxx Halperin
Date: June 15, 2025
From: Dr. L. yyy, MD
To: Nephrology Department
Subject: Referral for advanced evaluation of chronic kidney disease in the context of complex
multi-system comorbidities.
Dear Colleague,
I am referring my long-standing patient, Mr. xxx W. Halperin, a 71-year-old retired school
principal, for nephrology evaluation in light of accelerated decline in renal function, persistent' ---
[{'event_type': 'Referral Sent', 'date': '2025-06-15', 'provider': 'Dr. L. yyy, MD', 'details': 'Referral for nephrology evaluation of Mr. xxx W. Halperin for advanced evaluation of chronic kidney disease'}]</s>
Extract all events and their corresponding dates from the paragraph below.
Return the output as a JSON array with two fields: 'event' and 'date'.
Use concise phrasing. Include only entries that have a valid date.

Paragraph:
Referral Letter: Nephrology Evaluation for 

In [ ]:
import os
import torch
from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import TextStreamer

# ---- paths ----
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"   # base instruct model
ADAPTER_DIR = "/content/drive/MyDrive/HealthcareCJM/TuningPart2/Mistral_Trained_Data/checkpoint-60/"

# ---- config ----
max_seq_length = 2048
dtype = None          # auto
load_in_4bit = True   # QLoRA inference

# ---- load ----
def load_trained_model(adapter_dir: str):
    """
    Loads the fine-tuned model. If `adapter_config.json` exists in adapter_dir,
    we assume LoRA-only weights and attach them to the base model.
    Otherwise, try to load a full model from adapter_dir.
    """
    adapter_cfg = os.path.join(adapter_dir, "adapter_config.json")
    if os.path.exists(adapter_cfg):
        # 1) load base in 4-bit
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name     = BASE_MODEL,
            max_seq_length = max_seq_length,
            dtype          = dtype,
            load_in_4bit   = load_in_4bit,
        )
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # 2) attach LoRA adapter
        model = PeftModel.from_pretrained(model, adapter_dir)
        # (optional) enable Unsloth inference fast-path
        try:
            FastLanguageModel.for_inference(model)
        except:
            pass
        return model, tokenizer
    else:
        # Try to load a fully saved model from the directory
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name     = adapter_dir,
            max_seq_length = max_seq_length,
            dtype          = dtype,
            load_in_4bit   = load_in_4bit,
        )
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        try:
            FastLanguageModel.for_inference(model)
        except:
            pass
        return model, tokenizer

model, tokenizer = load_trained_model(ADAPTER_DIR)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Mistral patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
import json
import pandas as pd
import torch
from datetime import datetime
import time

# ---------- Your exact system prompt ----------
mistral_system_prompt = """
You are an AI system specialized in extracting structured clinical events for Healthcare Journey Mapping.

Your goal: From the given text chunk, extract all clinical events mentioned and represent them in a structured, chronological manner.

### Instructions:
1. Identify events such as:
   - Diagnosis
   - Referral Sent
   - Specialist Visit
   - Hospitalization
   - Procedure
   - Imaging Ordered / Imaging Result
   - Lab Ordered / Lab Result
   - Medication Started / Changed / Stopped
   - Follow-up Visit
   - Missed Appointment
   - Patient Reported Symptom
   - Functional/Social Note
   - Other relevant clinical events

2. For each event, provide:
   - **event_type**: One of the categories above.
   - **date**: Use `YYYY-MM-DD` if exact date is available; otherwise `YYYY-MM` or "Unknown".
   - **provider**: Doctor name or facility if present; otherwise "N/A".
   - **details**: Short summary (e.g., reason, findings, medication name/dose, lab value).

3. Do NOT infer or hallucinate events not explicitly mentioned in the text.
4. Keep identifiers exactly as in text (do not generate new names).
5. Output strictly as a JSON array.

### JSON Format:
[
  {
    "event_type": "",
    "date": "",
    "provider": "",
    "details": ""
  }
]

### Example:
[
  {
    "event_type": "Referral Sent",
    "date": "2023-03-01",
    "provider": "Dr. Smith",
    "details": "Concern: nephropathy"
  },
  {
    "event_type": "Hospitalization",
    "date": "2024-02-10",
    "provider": "St. Mary’s Hospital",
    "details": "Acute kidney injury, creatinine peaked at 3.0 mg/dL"
  }
]

Now process this text chunk and return ONLY valid JSON with all events found:
<<CHUNK>>
"""

# ---------- Config ----------
num_chunks_to_process = len(page_chunks)   # or set a smaller number
# num_chunks_to_process = 3
rows = []
output_file = Model_Output_AF_Tuned



# ---------- Helpers ----------
def build_full_prompt(chunk_text: str) -> str:
    return (
        mistral_system_prompt.replace("<<CHUNK>>", chunk_text)
        + "\n\nReturn ONLY a valid JSON array without any text before or after."
    )

def as_raw_text_cell(chunk) -> str:
    # Match your example: Raw Text should begin with page_content='...'
    text = getattr(chunk, "page_content", str(chunk))
    return f"page_content='{text}'"

def extract_json_array(generated: str) -> str:
    """
    Extract the first top-level JSON array from a text string.
    Returns the raw JSON string (pretty printed) or raises.
    """
    cleaned = generated.strip()

    # strip optional code fences
    if cleaned.startswith("```json"):
        cleaned = cleaned[len("```json"):].lstrip()
    if cleaned.endswith("```"):
        cleaned = cleaned[:-3].rstrip()

    start = cleaned.find('[')
    end = cleaned.rfind(']') + 1
    if start == -1 or end <= 0 or start >= end:
        raise ValueError("No JSON array found in model output.")

    # validate JSON & pretty print
    obj = json.loads(cleaned[start:end])
    return json.dumps(obj, ensure_ascii=False, indent=2)

# ---------- Main loop ----------
device = next(model.parameters()).device

for i, chunk in enumerate(page_chunks[:num_chunks_to_process], start=1):
    # Build full prompt inside a single "user" message (works with Mistral chat template)
    full_prompt = build_full_prompt(getattr(chunk, "page_content", str(chunk)))
    messages = [{"role": "user", "content": full_prompt}]

    # Tokenize according to the tokenizer's chat template
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(device)

    t0 = time.perf_counter()
    # Generate deterministically for clean JSON
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs,
            max_new_tokens=1024,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True,
        )
    elapsed_ms = (time.perf_counter() - t0) * 1000.0

    # Slice to only the newly generated tokens (exclude the prompt)
    gen_only_ids = output_ids[0, inputs.shape[-1]:]
    gen_text = tokenizer.decode(gen_only_ids, skip_special_tokens=True)

    # Extract and pretty-print the JSON array; if it fails, keep raw text
    try:
        pretty_json = extract_json_array(gen_text)
        output_cell = pretty_json
    except Exception:
        # Fallback to raw model output if JSON parsing fails
        output_cell = gen_text.strip()

    rows.append({
        "S.No": i,
        "Raw Text": as_raw_text_cell(chunk),
        "Output": output_cell,
        "Latency": elapsed_ms,  # <--- NEW
    })

    # Print the results
    print(f"--- Chunk {i} ---")
    print(f"Raw Text: {as_raw_text_cell(chunk)}")
    print(f"Output: {output_cell}")
    print(f"latency_ms: {elapsed_ms}")
    print("-" * 20)

# ---------- Save exactly 3 columns to Excel ----------
print("Saving results to Excel...")
df = pd.DataFrame(rows, columns=["S.No", "Raw Text", "Output", "Latency"])
df.to_excel(output_file, index=False)
print(f"✅ Saved {len(df)} rows to {output_file}")
print("Processing complete.")

--- Chunk 1 ---
Raw Text: page_content='Referral Letter: Nephrology Evaluation for Mr. xxx Halperin
Date: June 15, 2025
From: Dr. L. yyy, MD
To: Nephrology Department
Subject: Referral for advanced evaluation of chronic kidney disease in the context of complex
multi-system comorbidities.
Dear Colleague,
I am referring my long-standing patient, Mr. xxx W. Halperin, a 71-year-old retired school
principal, for nephrology evaluation in light of accelerated decline in renal function, persistent'
Output: [{'event_type': 'Referral Sent', 'date': '2025-06-15', 'provider': 'Dr. L. yyy, MD', 'details': 'Referral for nephrology evaluation of chronic kidney disease'}]
latency_ms: 14328.04591499962
--------------------
--- Chunk 2 ---
Raw Text: page_content='principal, for nephrology evaluation in light of accelerated decline in renal function, persistent
proteinuria, and increasing clinical complexity. Mr. XXXHalperinhas been under my care at
Lakeside Family Medicine since 2012 and presents a mult

Evaluation


In [ ]:
!pip -q install pandas openpyxl datasets

In [ ]:
# UPDATE THESE if your files live elsewhere in Drive
BF_XLSX = "/content/drive/MyDrive/HealthcareCJM/TuningPart2/Model_Output_BF_Tuned.xlsx"
AF_XLSX = "/content/drive/MyDrive/HealthcareCJM/TuningPart2/Model_Output_AF_Tuned.xlsx"

# Where to save the evaluation workbook
EVAL_XLSX = "/content/drive/MyDrive/HealthcareCJM/Evaluation_Report.xlsx"
Clinical_GT_Data = '/content/drive/MyDrive/HealthcareCJM/TuningPart2/Clinical_GT_Data.jsonl'

LATENCY_COL = "Latency"




In [ ]:
import json, ast, re, numpy as np, pandas as pd
from typing import List, Dict, Any, Tuple
from collections import Counter

ALLOWED_EVENT_TYPES = {
    "Diagnosis","Referral Sent","Specialist Visit","Hospitalization","Procedure",
    "Imaging Ordered","Imaging Result","Lab Ordered","Lab Result",
    "Medication Started","Medication Changed","Medication Stopped",
    "Follow-up Visit","Missed Appointment","Patient Reported Symptom",
    "Functional/Social Note","Other"
}

DATE_PATTERNS = [
    re.compile(r"^\d{4}-\d{2}-\d{2}$"),
    re.compile(r"^\d{4}-\d{2}$"),
    re.compile(r"^\d{4}$"),
]

def pick_cols(df: pd.DataFrame) -> Tuple[str,str]:
    if {"Raw Text","Output"}.issubset(df.columns): return "Raw Text","Output"
    if {"chunk_text","model_output"}.issubset(df.columns): return "chunk_text","model_output"
    raw_col = df.columns[1] if len(df.columns)>1 else df.columns[0]
    out_col = df.columns[-1]
    return raw_col, out_col

def parse_json_array_robust(text: str) -> Tuple[bool, List[Dict[str, Any]], str]:
    """Extract first [...] then try JSON -> Excel-unescape -> ast.literal_eval."""
    if text is None: return False, [], "None text"
    s = str(text).strip()
    if s.startswith("```json"): s = s[len("```json"):].lstrip()
    if s.endswith("```"): s = s[:-3].rstrip()
    i, j = s.find("["), s.rfind("]")+1
    if i==-1 or j<=0 or i>=j: return False, [], "No JSON array found"
    chunk = s[i:j]

    # JSON
    try:
        arr = json.loads(chunk)
        if isinstance(arr, list): return True, [e for e in arr if isinstance(e, dict)], ""
    except Exception as e1:
        err1 = str(e1)

    # Excel-quoted
    chunk2 = chunk
    if chunk2.startswith('"') and chunk2.endswith('"'): chunk2 = chunk2[1:-1]
    chunk2 = chunk2.replace('""','"')
    try:
        arr = json.loads(chunk2)
        if isinstance(arr, list): return True, [e for e in arr if isinstance(e, dict)], ""
    except Exception as e2:
        err2 = str(e2)

    # Python literal
    try:
        arr = ast.literal_eval(chunk2)
        if isinstance(arr, list):
            evs = [e for e in arr if isinstance(e, dict)]
            # normalize scalar types to strings where needed
            for e in evs:
                for k in ("event_type","date","provider","details"):
                    if k in e and not isinstance(e[k], str):
                        e[k] = str(e[k])
            return True, evs, ""
    except Exception as e3:
        err3 = str(e3)

    return False, [], f"json loads: {err1}; excel json: {err2}; literal_eval: {err3}"

def is_valid_date_format(date_str: str) -> bool:
    if not isinstance(date_str, str): return False
    s = date_str.strip()
    if s.lower() == "unknown": return True
    if "00" in s: return False
    for pat in DATE_PATTERNS:
        if pat.match(s):
            if len(s)==10:
                try:
                    y,m,d = map(int, s.split("-"))
                    return 1<=m<=12 and 1<=d<=31
                except: return False
            if len(s)==7:
                try:
                    y,m = map(int, s.split("-"))
                    return 1<=m<=12
                except: return False
            return True
    return False

# --- Provider matching ---
_token_re = re.compile(r"[a-z0-9]+")

def provider_overlap_strict(provider: str, raw_text: str) -> float:
    """Strict substring proxy: 1 if provider is in raw text, else 0. N/A -> NaN."""
    if not isinstance(provider, str) or provider.strip()=="" or provider=="N/A":
        return np.nan
    return float(provider.lower().strip() in str(raw_text).lower())

def provider_overlap_relaxed(provider: str, raw_text: str) -> float:
    """Token recall proxy: (# provider tokens found in raw) / (# provider tokens). N/A -> NaN."""
    if not isinstance(provider, str) or provider.strip()=="" or provider=="N/A":
        return np.nan
    p_toks = set(_token_re.findall(provider.lower()))
    if not p_toks: return np.nan
    raw_toks = set(_token_re.findall(str(raw_text).lower()))
    inter = len(p_toks & raw_toks)
    return inter / len(p_toks)

# --- Latency ---
def latency_stats(df: pd.DataFrame, raw_col: str, latency_col: str):
    if latency_col is None or latency_col not in df.columns:
        return dict(p50_ms_per_1k=np.nan, p95_ms_per_1k=np.nan, docs_per_min=np.nan)
    L = df[raw_col].astype(str).str.len().replace(0, np.nan)
    ms = pd.to_numeric(df[latency_col], errors="coerce")
    ms_per_1k = ms / (L/1000.0)
    p50 = float(np.nanpercentile(ms_per_1k, 50)) if ms_per_1k.notna().any() else np.nan
    p95 = float(np.nanpercentile(ms_per_1k, 95)) if ms_per_1k.notna().any() else np.nan
    docs_per_min = 60000.0 / float(np.nanmedian(ms)) if ms.notna().any() else np.nan
    return dict(p50_ms_per_1k=p50, p95_ms_per_1k=p95, docs_per_min=docs_per_min)


Unsupervised evaluation (no gold needed)

In [ ]:
def analyze_file(xlsx_path: str, latency_col: str):
    df = pd.read_excel(xlsx_path)
    raw_col, out_col = pick_cols(df)

    per_doc = []
    for idx, r in df.iterrows():
        raw = r.get(raw_col, "")
        out = r.get(out_col, "")
        ok, evs, err = parse_json_array_robust(out)

        # per-event booleans
        et_ok = [(e.get("event_type") in ALLOWED_EVENT_TYPES) for e in evs]
        dt_ok = [is_valid_date_format(e.get("date","")) for e in evs]
        prov_strict = [provider_overlap_strict(e.get("provider",""), raw) for e in evs]
        prov_relax  = [provider_overlap_relaxed(e.get("provider",""), raw) for e in evs]

        per_doc.append({
            "row_id": idx+1,
            "json_valid": ok,
            "n_events": len(evs),
            "event_type_in_allowed_rate": (np.nanmean(et_ok) if et_ok else np.nan),
            "date_format_valid_rate": (np.nanmean(dt_ok) if dt_ok else np.nan),
            "provider_overlap_strict": (np.nanmean(prov_strict) if prov_strict else np.nan),
            "provider_overlap_relaxed": (np.nanmean(prov_relax)  if prov_relax  else np.nan),
            "raw_text": raw,
            "parse_error": "" if ok else err,
        })

    details = pd.DataFrame(per_doc)

    # Summary (unsupervised)
    summary = {
        "n_docs": int(len(details)),
        "json_valid_rate": float(np.mean(details["json_valid"])) if len(details) else 0.0,
        "avg_events_per_doc": float(np.nanmean(details["n_events"])) if len(details) else 0.0,
        "event_type_allowed_mean": float(np.nanmean(details["event_type_in_allowed_rate"])),
        "date_format_valid_mean": float(np.nanmean(details["date_format_valid_rate"])),
        "provider_strict_mean": float(np.nanmean(details["provider_overlap_strict"])),
        "provider_relaxed_mean": float(np.nanmean(details["provider_overlap_relaxed"])),
    }

    # Latency (normalized)
    lat = latency_stats(df, raw_col, latency_col)
    summary.update(lat)
    return details, summary

bf_details, bf_summary = analyze_file(BF_XLSX, LATENCY_COL)
af_details, af_summary = analyze_file(AF_XLSX, LATENCY_COL)

summary_df = pd.DataFrame([
    {"Metric":"JSON Validity Rate",          "Before": round(bf_summary["json_valid_rate"]*100,2),      "After": round(af_summary["json_valid_rate"]*100,2),      "Unit":"%"},
    {"Metric":"Avg Events per Doc",          "Before": round(bf_summary["avg_events_per_doc"],2),       "After": round(af_summary["avg_events_per_doc"],2),       "Unit":"events"},
    {"Metric":"Event Type Allowed (mean)",   "Before": round(bf_summary["event_type_allowed_mean"]*100,2), "After": round(af_summary["event_type_allowed_mean"]*100,2), "Unit":"% events"},
    {"Metric":"Date Format Valid (mean)",    "Before": round(bf_summary["date_format_valid_mean"]*100,2),  "After": round(af_summary["date_format_valid_mean"]*100,2),  "Unit":"% events"},
    {"Metric":"Provider Overlap (strict)",   "Before": round(bf_summary["provider_strict_mean"]*100,2),    "After": round(af_summary["provider_strict_mean"]*100,2),    "Unit":"% events"},
    {"Metric":"Provider Overlap (relaxed)",  "Before": round(bf_summary["provider_relaxed_mean"]*100,2),   "After": round(af_summary["provider_relaxed_mean"]*100,2),   "Unit":"% events"},
    {"Metric":"Latency P50 (ms / 1k chars)", "Before": bf_summary["p50_ms_per_1k"],                      "After": af_summary["p50_ms_per_1k"],                      "Unit":"ms/1k"},
    {"Metric":"Latency P95 (ms / 1k chars)", "Before": bf_summary["p95_ms_per_1k"],                      "After": af_summary["p95_ms_per_1k"],                      "Unit":"ms/1k"},
    {"Metric":"Docs/min on 1 GPU",           "Before": bf_summary["docs_per_min"],                       "After": af_summary["docs_per_min"],                       "Unit":"docs/min"},
])

with pd.ExcelWriter(EVAL_XLSX, engine="openpyxl") as w:
    summary_df.to_excel(w, sheet_name="summary", index=False)
    bf_details.to_excel(w, sheet_name="bf_details", index=False)
    af_details.to_excel(w, sheet_name="af_details", index=False)

print("Saved:", EVAL_XLSX)
summary_df

Saved: /content/drive/MyDrive/HealthcareCJM/Evaluation_Report.xlsx


/tmp/ipython-input-2685497234.py:23: RuntimeWarning: Mean of empty slice
  "provider_overlap_strict": (np.nanmean(prov_strict) if prov_strict else np.nan),
/tmp/ipython-input-2685497234.py:24: RuntimeWarning: Mean of empty slice
  "provider_overlap_relaxed": (np.nanmean(prov_relax)  if prov_relax  else np.nan),
/tmp/ipython-input-2685497234.py:23: RuntimeWarning: Mean of empty slice
  "provider_overlap_strict": (np.nanmean(prov_strict) if prov_strict else np.nan),
/tmp/ipython-input-2685497234.py:24: RuntimeWarning: Mean of empty slice
  "provider_overlap_relaxed": (np.nanmean(prov_relax)  if prov_relax  else np.nan),


,Metric,Before,After,Unit
0,JSON Validity Rate,100.000000,90.480000,%
1,Avg Events per Doc,4.930000,0.900000,events
2,Event Type Allowed (mean),83.650000,100.000000,% events
3,Date Format Valid (mean),66.860000,94.740000,% events
4,Provider Overlap (strict),68.180000,100.000000,% events
5,Provider Overlap (relaxed),79.550000,100.000000,% events
6,Latency P50 (ms / 1k chars),60513.700124,9362.229882,ms/1k
7,Latency P95 (ms / 1k chars),114343.068704,30447.506465,ms/1k
8,Docs/min on 1 GPU,2.107427,14.267724,docs/min


Gold Json

In [ ]:
if Clinical_GT_Data:
    def normalize_date_match(pred, gold):
        if not isinstance(pred, str) or not isinstance(gold, str): return False
        p, g = pred.strip().lower(), gold.strip().lower()
        if p == g: return True
        if re.match(r"^\d{4}-\d{2}$", g) and re.match(r"^\d{4}-\d{2}-\d{2}$", p): return p[:7]==g
        if re.match(r"^\d{4}-\d{2}$", p) and re.match(r"^\d{4}-\d{2}-\d{2}$", g): return g[:7]==p
        return False

    def provider_token_overlap(pred, gold):
        pt = set(re.findall(r"[a-z0-9]+", (pred or "").lower()))
        gt = set(re.findall(r"[a-z0-9]+", (gold or "").lower()))
        if not gt: return 1.0 if not pt else 0.0
        return len(pt & gt) / max(1, len(gt))

    def event_type_f1_micro(gold_events, pred_events):
        g = Counter([e.get("event_type") for e in gold_events if e.get("event_type") in ALLOWED_EVENT_TYPES])
        p = Counter([e.get("event_type") for e in pred_events if e.get("event_type") in ALLOWED_EVENT_TYPES])
        tp = sum(min(g[t], p[t]) for t in ALLOWED_EVENT_TYPES)
        fp = sum(max(0, p[t] - min(g[t], p[t])) for t in ALLOWED_EVENT_TYPES)
        fn = sum(max(0, g[t] - min(g[t], p[t])) for t in ALLOWED_EVENT_TYPES)
        prec = tp/(tp+fp) if (tp+fp)>0 else 0.0
        rec  = tp/(tp+fn) if (tp+fn)>0 else 0.0
        return 0.0 if (prec+rec)==0 else 2*prec*rec/(prec+rec)

    def safe_extract_preds_from_excel(xlsx):
        df = pd.read_excel(xlsx)
        raw_col, out_col = pick_cols(df)
        rows = []
        for _, r in df.iterrows():
            raw = str(r.get(raw_col,""))
            ok, evs, _ = parse_json_array_robust(r.get(out_col,""))
            rows.append({"input": raw, "pred": evs if ok else []})
        return pd.DataFrame(rows)

    # Load gold
    gold_rows = []
    with open(Clinical_GT_Data, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                gold_rows.append(json.loads(line))
    gold_df = pd.DataFrame(gold_rows)   # input, output

    def eval_supervised_one(pred_xlsx):
        pred_df = safe_extract_preds_from_excel(pred_xlsx)
        merged = gold_df.merge(pred_df, on="input", how="inner")
        rows = []
        for _, r in merged.iterrows():
            ge, pe = r["output"] or [], r["pred"] or []
            f1 = event_type_f1_micro(ge, pe)
            used = set(); hits = 0; total = 0; provs = []
            for g_ev in ge:
                et = g_ev.get("event_type")
                match = None
                for i, p_ev in enumerate(pe):
                    if i in used: continue
                    if p_ev.get("event_type")==et:
                        match = i; break
                if match is None: continue
                used.add(match)
                p_ev = pe[match]
                total += 1
                if normalize_date_match(p_ev.get("date",""), g_ev.get("date","")):
                    hits += 1
                provs.append(provider_token_overlap(p_ev.get("provider",""), g_ev.get("provider","")))
            rows.append({
                "event_f1_micro": f1,
                "date_exact_match": (hits/total) if total>0 else np.nan,
                "provider_overlap_mean": float(np.mean(provs)) if provs else np.nan,
            })
        return pd.DataFrame(rows)

    bf_sup = eval_supervised_one(BF_XLSX)
    af_sup = eval_supervised_one(AF_XLSX)

    with pd.ExcelWriter(EVAL_XLSX, engine="openpyxl", mode="a", if_sheet_exists="overlay") as w:
        bf_sup.to_excel(w, sheet_name="bf_supervised", index=False)
        af_sup.to_excel(w, sheet_name="af_supervised", index=False)

    print("Supervised sheets appended:", EVAL_XLSX)


Supervised sheets appended: /content/drive/MyDrive/HealthcareCJM/Evaluation_Report.xlsx
